In [ ]:
!nvidia-smi

In [ ]:
import os

# 1. Clone Repository (Safe to re-run)
if not os.path.exists("InfiniteTalk"):
    !git clone https://github.com/MeiGen-AI/InfiniteTalk
else:
    print("Repository already cloned.")

# 2. Enter Directory (Required after every restart)
%cd InfiniteTalk

# 3. Manage Disk Space & Dependencies
print("Cleaning up unused libraries to save space...")
# Uninstall heavy unused pre-installed libraries to free up ~5-8GB
!pip uninstall -y tensorflow jax jaxlib
# Uninstall existing pytorch to prevent disk spike during upgrade
!pip uninstall -y torch torchvision torchaudio

print("Installing dependencies... (This may take a few minutes)")

# Install PyTorch and xformers (with --no-cache-dir)
!pip install --no-cache-dir torch==2.4.1 torchvision==0.19.1 torchaudio==2.4.1 --index-url https://download.pytorch.org/whl/cu121
!pip install --no-cache-dir -U xformers==0.0.28 --index-url https://download.pytorch.org/whl/cu121

# Install Flash Attention and dependencies
!pip install --no-cache-dir misaki[en] ninja psutil packaging wheel
!pip install --no-cache-dir flash_attn==2.7.4.post1

# Install other dependencies
!pip install --no-cache-dir -r requirements.txt
!pip install --no-cache-dir librosa huggingface_hub

# Install system dependencies and clean up
!apt-get update && apt-get install -y ffmpeg && apt-get clean && rm -rf /var/lib/apt/lists/*

In [ ]:
import os
os.makedirs("weights", exist_ok=True)

# Download base model and InfiniteTalk weights
!hf download Wan-AI/Wan2.1-I2V-14B-480P --local-dir ./weights/Wan2.1-I2V-14B-480P
!hf download MeiGen-AI/InfiniteTalk --local-dir ./weights/InfiniteTalk

# Download English audio encoder (DEFAULT)
!hf download facebook/wav2vec2-base-960h --local-dir ./weights/wav2vec2-base-960h

# Alternative: For Chinese audio support, uncomment the lines below:
# !hf download TencentGameMate/chinese-wav2vec2-base --local-dir ./weights/chinese-wav2vec2-base
# !hf download TencentGameMate/chinese-wav2vec2-base model.safetensors --revision refs/pr/1 --local-dir ./weights/chinese-wav2vec2-base

In [ ]:
# Enable public link for Gradio
!sed -i 's/demo.launch(server_name="0.0.0.0", debug=True, server_port=8418)/demo.launch(server_name="0.0.0.0", debug=True, server_port=8418, share=True)/' app.py

# Launch Gradio interface with English audio encoder (DEFAULT)
!python app.py --ckpt_dir weights/Wan2.1-I2V-14B-480P --wav2vec_dir 'weights/wav2vec2-base-960h' --infinitetalk_dir weights/InfiniteTalk/single/infinitetalk.safetensors --num_persistent_param_in_dit 0 --motion_frame 9

# Alternative: For Chinese audio, uncomment below and comment the English line above:
# !python app.py --ckpt_dir weights/Wan2.1-I2V-14B-480P --wav2vec_dir 'weights/chinese-wav2vec2-base' --infinitetalk_dir weights/InfiniteTalk/single/infinitetalk.safetensors --num_persistent_param_in_dit 0 --motion_frame 9